
#            Calcul numérique du MGI d'un RRR par PNL avec contraintes #



|![alternative text](robot-rrr3-sol.png)|
|:--:| 
| *  Robot RRR  * |

## Objectif
On se propose de calculer de manière numérique un modèle géométrique inverse (MGI) d'un robot RRR en utilisant les fonctions de $scipy$ $optimize$ et en considérant des contraintes.

* La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.
* La situation de l'outil est défini par le vecteur $X = (x, y, \theta)^t$ .


## Calcul du MGD
La fonction suivante calcule le modèle géométrique direct:


In [21]:
import numpy as np

'''
Cartesian displacement calculator (MGD - Direct Geometric Model) for an RRR robot (3 degrees of freedom with rotary joints).
The MGD takes input joint configuration in degrees and returns the Cartesian coordinates (x, y, z) of the robot's end-effector in space.
# INPUT: q = configuration vector (deg, deg, deg)
# OUTPUT: Xc = pose vector = (x, y, z)
'''
def mgd(qdeg):
    # Parameters of the robot
    a, b, c = 1, 1, 1                   # Lengths of the robot arms
    qrad = np.deg2rad(qdeg)             # Convert input joint angles from degrees to radians
    c1 = np.cos(qrad[0])                # Cosine of the first joint angle
    s1 = np.sin(qrad[0])                # Sine of the first joint angle
    c23 = np.cos(qrad[2] + qrad[1])     # Cosine of the sum of second and third joint angles
    s23 = np.sin(qrad[2] + qrad[1])     # Sine of the sum of second and third joint angles
    c2 = np.cos(qrad[1])                # Cosine of the second joint angle
    s2 = np.sin(qrad[1])                # Sine of the second joint angle

    # Calculate the Cartesian coordinates (x, y, z) of the robot's end-effector
    x = a * c1 * c2 + b * c1 * c23
    y = a * s1 * c2 + b * s1 * c23
    z = a * s2 + b * s23 + c

    # Create a NumPy array to store the Cartesian coordinates
    Xd = np.asarray([x, y, z])

    # Return the Cartesian coordinates of the end-effector
    return Xd

In [22]:
import numpy as np

''' 
Calculates the analytical Jacobian matrix J(q) for an RRR robot, which relates the end-effector's 
linear and angular velocities to the joint velocities. It takes input joint angles in degrees, 
converts them to radians, and then computes the Jacobian matrix based on the robot's parameters and the joint angles. 
The result is returned as a NumPy array representing the analytical Jacobian matrix.
'''
def jacobian(qdeg):
    # Parameters of the robot
    a, b = 1, 1                         # Lengths of the robot arms
    qrad = np.deg2rad(qdeg)             # Convert input joint angles from degrees to radians
    c1 = np.cos(qrad[0])                # Cosine of the first joint angle
    s1 = np.sin(qrad[0])                # Sine of the first joint angle
    c23 = np.cos(qrad[2] + qrad[1])     # Cosine of the sum of second and third joint angles
    s23 = np.sin(qrad[2] + qrad[1])     # Sine of the sum of second and third joint angles
    c2 = np.cos(qrad[1])                # Cosine of the second joint angle
    s2 = np.sin(qrad[1])                # Sine of the second joint angle

    # Calculate the analytical Jacobian matrix J(q) for the RRR robot
    Ja = np.array([[-a * s1 * c2 - b * s1 * c23, -a * c1 * s2 - b * c1 * s23, -b * c1 * s23],
                   [a * c1 * c2 + b * c1 * c23, -a * s1 * s2 - b * s1 * s23, -b * s1 * s23],
                   [0, a * c2 + b * c23, b * c23]])

    # Return the analytical Jacobian matrix J(q)
    return Ja

## 1 - Calcul du MGI sans contraintes

Trouver et utiliser les fonctions de $optimize$ permettant de calculer le MGI $(root, broyden, minimize,...)$.  
Comparer les résultats avec votre implémentation


In [23]:
import numpy as np
from scipy.optimize import minimize, root

'''
Performs optimization and root-finding to find the joint angles that achieve a desired end-effector position (Xdesired) for the RRR robot. 
It uses the provided cost functions to formulate and solve the optimization and root-finding problems, respectively. 
Finally, it prints the results, including the joint angles and the corresponding end-effector positions.
'''

# Define a cost function for the root-finding method
def cost_root(q):
    return mgd(q) - Xdesired

# Define a cost function for the optimization method
def cost_function(q, Xdesired):
    Xcurrent = mgd(q)
    return np.linalg.norm(Xcurrent - Xdesired)

qinit = np.asarray([5.0, 0, 4.0])       # Initial guess for joint angles
qbut = np.asarray([-45.0, 0, 45.0])     # Target joint angles
Xdesired = mgd(qbut)                    # Calculate the desired end-effector position based on the target joint angles

# Use the optimization method (BFGS) to find the joint angles that minimize the distance to the target
result = minimize(cost_function, qinit, args=(Xdesired,), method='BFGS')
q_solution_mini = result.x              # Extract the optimized joint angles
X_solution_mini = mgd(q_solution_mini)  # Calculate the end-effector position for the optimized joint angles

# Use the root-finding method (hybr) to find joint angles that satisfy the root cost function
sol = root(cost_root, qinit, jac=jacobian, method="hybr")
q_solution_root = sol.x                 # Extract the root-finding result joint angles
X_solution_root = mgd(q_solution_root)  # Calculate the end-effector position for the root-finding result joint angles

# Use the root-finding method (broyden1) to find joint angles that satisfy the root cost function
sol1 = root(cost_root, qinit, method="broyden1")
q_solution_broy = sol1.x                # Extract the root-finding result joint angles
X_solution_broy = mgd(q_solution_broy)  # Calculate the end-effector position for the root-finding result joint angles

# Print the results
print("Coordinates q_solution_mini:", q_solution_mini)
print("Coordinates X_solution_mini:", X_solution_mini)
print(result.success)
print("\r")
print("Coordinates q_solution_root:", q_solution_root)
print("Coordinates X_solution_root:", X_solution_root)
print(sol.success)
print("\r")
print("Coordinates q_solution_broy:", q_solution_broy % 360)  # Ensure joint angles are within [0, 360) degrees
print("Coordinates X_solution_broy:", X_solution_broy)
print(sol1.success)
print("\r")
print("X desired =", Xdesired[0], "Y desired =", Xdesired[1], "Z desired =", Xdesired[2])

Coordinates q_solution_mini: [-4.50000000e+01 -2.08840734e-08  4.50000000e+01]
Coordinates X_solution_mini: [ 1.20710678 -1.20710678  1.70710678]
False

Coordinates q_solution_root: [-4.50000000e+01  1.44802271e-09  4.50000000e+01]
Coordinates X_solution_root: [ 1.20710678 -1.20710678  1.70710678]
True

Coordinates q_solution_broy: [315.00000021  45.00000151 314.99999366]
Coordinates X_solution_broy: [ 1.20710677 -1.20710676  1.70710672]
True

X desired = 1.2071067811865477 Y desired = -1.2071067811865477 Z desired = 1.7071067811865475


**Conclusion**  
Comme on peut le voir en utilisant les diverses méthodes on obtient des degrès de précision différent.  
*minimize* et *root* ont la même précision sur X_solution cependant root semble plus précis sur les mouvements de q.  
En comparaison Broyden est moins précis sur X_solution et surtout ,sans contraintes, présentent des valeurs de q trop élevé.

**Note** : Le success renvoie le résultat false pour minimize car nous évoluons de R3 vers R3 et non pas de Rn vers R

## 2 - Calcul du MGI avec  contraintes

Définir et utiliser une fonction $fun$ permettant de calculer la solution du MGI la plus proche de la configuration initiale $qinit$ donnée à la fonction $minimize$.

$res = minimize(fun, qinit, ....)$




In [24]:
import numpy as np
from scipy.optimize import minimize

'''
Finds the joint angles that minimize the distance between the initial joint angles (qinit) and the target joint angles (qbut) 
while ensuring that the end-effector reaches the desired position (Xbut, Ybut, Zbut). 
It uses an objective function that calculates the Euclidean distance between joint angle vectors. 
The code sets up equality constraints to enforce the desired end-effector position. 
Finally, it uses the 'slsqp' optimization method to find the optimized joint angles and prints the results.
'''

# Define an objective function that calculates the distance between qdeg and qinit
def objective_distance(qdeg):
    return np.linalg.norm(qdeg - qinit)

qbut = np.asarray([-45.0, 0, 45.0])     # Target joint angles
Xbut = mgd(qbut)                        # Calculate the desired end-effector position based on the target joint angles
print("Xbut =", Xbut[0], "Ybut =", Xbut[1], "Zbut =", Xbut[2])

qinit = np.asarray([60.0, 45, 8.0])     # Initial guess for joint angles
Xinit = mgd(qinit)                      # Calculate the initial end-effector position based on the initial joint angles
print("Xinit =", Xinit[0], "Yinit =", Xinit[1], "Zinit =", Xinit[2])

# Define equality constraints to ensure that the end-effector reaches the desired position (Xbut, Ybut, Zbut)
constraints = ({'type': 'eq', 'fun': lambda qdeg: Xbut[0] - mgd(qdeg)[0]},
               {'type': 'eq', 'fun': lambda qdeg: Xbut[1] - mgd(qdeg)[1]},
               {'type': 'eq', 'fun': lambda qdeg: Xbut[2] - mgd(qdeg)[2]})

# Use the optimization method (slsqp) to find the joint angles that minimize the objective function
solMinimize = minimize(objective_distance, qinit, constraints=constraints, method='slsqp')
print(solMinimize.x)                    # Extract the optimized joint angles
print(mgd(solMinimize.x))               # Calculate the end-effector position for the optimized joint angles
print(solMinimize.success)              # Print the optimization success status

Xbut = 1.2071067811865477 Ybut = -1.2071067811865477 Zbut = 1.7071067811865475
Xinit = 0.6544609021692981 Yinit = 1.1335595341245885 Zinit = 2.5057422912338403
[ -45.         -720.00000001 1485.00000002]
[ 1.20710678 -1.20710678  1.70710678]
True


**Conclusion**  
Ici on ajoute une contrainte d'égalité pour obtenir le mouvement minimum et trouver le point le plus proche, on a donc utilisé la méthode minimize en ajoutant un paramètre contraints qu'on a défini avec notre contrainte d'égalité.

## 3 - Calcul du MGI avec  contraintes

Définir et utiliser une fonction permettant de calculer la solution du MGI en prenant en compte les butées du robot.

$q_1 \in [-120, 120]$

$q_2 \in [-45, 120]$

$q_3 \in [-120, 120]$


In [25]:
import numpy as np
from scipy.optimize import minimize

'''
Performs inverse kinematics (MGI) for the 3-DOF robot. 
It finds the joint angles (q1, q2, q3) that achieve a specified target position (x, y, z) while respecting joint limits. 
The code defines the direct kinematics, objective function, and uses the minimize function from SciPy to find the joint angles. 
Finally, it prints the results or indicates if convergence failed.
'''

# Define joint limits
q1_min, q1_max = -120, 120
q2_min, q2_max = -45, 120
q3_min, q3_max = -120, 120

# Define the direct kinematics function for the robot
def direct_kinematics(q):
    q1, q2, q3 = q
    x = q1 + q2 + q3
    y = q1 - q2 + q3
    z = q1 + q2 - q3
    return np.array([x, y, z])

# Define the objective function to minimize the distance between the current position and the target position
def objective_function(q, target_position):
    current_position = direct_kinematics(q)
    return np.linalg.norm(target_position - current_position)

# Define a function to solve the inverse kinematics problem (MGI) and find joint angles for a target position
def solve_mgi(target_position):
    initial_guess = [0, 0, 0]
    bounds = [(q1_min, q1_max), (q2_min, q2_max), (q3_min, q3_max)]
    
    # Check if the target position is within joint limits
    if any(x < q_min or x > q_max for x, (q_min, q_max) in zip(target_position, bounds)):
        return None
    
    # Use the optimization method to find joint angles that minimize the objective function
    result = minimize(objective_function, initial_guess, args=(target_position,), bounds=bounds)
    
    # Check if optimization was successful
    if result.success:
        q1, q2, q3 = result.x
        return q1, q2, q3
    else:
        return None

# Specify a target position in Cartesian coordinates
target_position = np.array([10, 10, 10])

# Use the solve_mgi function to find the joint angles that achieve the target position
result = solve_mgi(target_position)

# Check if a solution was found
if result is not None:
    q1, q2, q3 = result
    print("MGI Solution: q1 =", q1, "q2 =", q2, "q3 =", q3)
else:
    print("Convergence failure.")

MGI Solution: q1 = 10.000000005299269 q2 = -2.2444141436898157e-09 q3 = -2.24440172938732e-09


**Conclusion**  
Ici on utilise la fonction minimize pour calculer le MGI en lui ajoutant des contraintes de limites sur q afin de limiter ses mouvements et ainsi correspondre aux restrictions sur les butées du robot.  
En conlusion il a de nombreuses possiblités d'ajout de contraintes pour correspondre au mieux à notre problème.  
Ici on défini :
* un **qmin** et un **qmax** pour définir les butées du robot
* une fonction **direct_kinematics** pour définir les mouvements sur x, y et z définissant la position du robot.  
* la méthode **minimize** pour récupérer les valeurs et on vérifie sur la convergence est possible ou non.